# CCI MERIS 4.1

## Initialisation

Loading modules required for the code below.

The CCI MERIS 4.1 observed area dataset is first masked using the land mask.

In [ ]:
import logging
import logging.config
import warnings

import matplotlib as mpl
from joblib import Parallel, delayed
from tqdm import tqdm

from wildfires.analysis.plotting import *
from wildfires.data.cube_aggregation import *
from wildfires.data.datasets import *
from wildfires.logging_config import JUPYTER_LOGGING as LOGGING
from wildfires.utils import get_ncpus, get_unmasked
from wildfires.utils import land_mask as get_land_mask
from wildfires.utils import match_shape

In [ ]:
logging.config.dictConfig(LOGGING)
warnings.filterwarnings("ignore", ".*Collapsing a non-contiguous coordinate.*")
warnings.filterwarnings("ignore", ".*DEFAULT_SPHERICAL_EARTH_RADIUS*")
mpl.rc("figure", figsize=(14, 6))

In [ ]:
monthly_obs, mean_obs, climatology_obs = prepare_selection(
    Datasets(CCI_BurnedArea_MERIS_4_1()).select_variables("fraction of observed area")
)

In [ ]:
land_mask = ~get_land_mask()
mean_obs.cube.data.mask |= match_shape(land_mask, mean_obs.cube.shape)

## Mean observed area fraction

In [ ]:
_ = cube_plotting(mean_obs.cube, title="Mean Fraction of Observed Area")

## Number of observations where observed area fraction >= 80%

In [ ]:
min_thres = 0.8
observed_counts = (monthly_obs.cube.data >= min_thres).astype("float64")
observed_counts.mask |= match_shape(land_mask, observed_counts.shape)
sum_obs_counts = np.ma.sum(observed_counts, axis=0)

In [ ]:
init_date = str(monthly_obs.cube.coord("time").cell(0).point).split()[0]
fin_date = str(monthly_obs.cube.coord("time").cell(-1).point).split()[0]
_ = cube_plotting(
    sum_obs_counts,
    title=f"Observed Months >= 80% ({init_date} to {fin_date})",
    label="counts",
    boundaries=[0, 2, 4, 20, 60, 75, 83],
)

## Before-after Comparison of Mean Burned Area

The effect of applying the observational fraction mask will be investigated.

In [ ]:
mean_ba = prepare_selection(
    Datasets(CCI_BurnedArea_MERIS_4_1()).select_variables("CCI MERIS BA"), which="mean"
).cube
monthly_vars = prepare_selection(
    Datasets(CCI_BurnedArea_MERIS_4_1()).select_variables(
        ("CCI MERIS BA", "fraction of observed area")
    ),
    which="monthly",
)
monthly_obs = monthly_vars.select_variables(
    "fraction of observed area", inplace=False
).cube
monthly_ba = monthly_vars.select_variables("CCI MERIS BA").cube

mean_ba.data.mask |= match_shape(land_mask, mean_ba.shape)
monthly_ba.data.mask |= match_shape(land_mask, monthly_ba.shape)

### No Masking

In [ ]:
mpl.rc("figure", figsize=(20, 10))
mean_ba_value = mean_ba.collapsed(
    ("latitude", "longitude"),
    iris.analysis.MEAN,
    weights=iris.analysis.cartography.area_weights(mean_ba),
).data
_ = cube_plotting(mean_ba, log=True, title=f"Mean CCI MERIS BA {mean_ba_value:0.3e}")

### With Observational Fraction Mask >= 80%

A substantially higher mean burned area is observed.

In [ ]:
mpl.rc("figure", figsize=(20, 10))
monthly_ba_masked = CCI_BurnedArea_MERIS_4_1.get_obs_masked_dataset(
    "CCI MERIS BA", thres=0.8
)["CCI MERIS BA"]
mean_ba_masked = monthly_ba_masked.collapsed("time", iris.analysis.MEAN)

masked_mean_ba_value = mean_ba_masked.collapsed(
    ("latitude", "longitude"),
    iris.analysis.MEAN,
    weights=iris.analysis.cartography.area_weights(mean_ba_masked),
).data

_ = cube_plotting(
    mean_ba_masked,
    log=True,
    title=f"Mean Masked CCI MERIS BA >= 80% Obs {masked_mean_ba_value:0.3e}",
)

### Determining Bin Sizing from Histogram of Values Above

We want to calculate the edges of bins such that the log map plot of mean burned area values is made more informative. This will be attempted by having data in each bin occupy a uniform proportion of values.

In [ ]:
mpl.rc("figure", figsize=(14, 6))

source_data = get_unmasked(mean_ba_masked.data)
non_zero_source = np.sort(source_data[source_data > 0])

plt.hist(non_zero_source, bins=100)
plt.ylabel("Occurrences")
plt.xlabel("Mean BA Fraction")

plt.yscale("log")
plt.xscale("log")

# Get percentiles. Ignore 0s in this process, and append them later on to the lowest bin.
# This is because 0s make up the predominant proportion of values.

bin_edges = [0.0] + list(np.percentile(non_zero_source, np.linspace(0, 100, 8)))

for bin_edge in bin_edges[1:]:
    plt.axvline(bin_edge, c="C4")

# Trying out the new bin edges.

mpl.rc("figure", figsize=(20, 10))
_ = cube_plotting(
    mean_ba_masked,
    log=True,
    title=f"Mean Masked CCI MERIS BA >= 80% Obs {masked_mean_ba_value:0.3e}",
    boundaries=bin_edges,
)